# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

Created a heat map diplaying the all cities from part-1 WeatherPy

Further narrowed locations based on weather conditions
   1. A max temperature lower than 80 degrees but higher than 70.
   2.  Wind speed less than 10 mph.
   3.  Zero cloudiness.

After locations being filtered, found hotel names for each city using google Place Search API to locate nearby first hotel within 5000 meters of the location coordinates (Latitudes and Longitudes). 

Thereafter, recreated dataframe adding hotel names column and plotted hotels location on the heatmap with pin marker displaying the Hotel name, city and Country, when click over the marker.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the file location
weatherpy_file = "../WeatherPy/Output/city_data.csv"

# read csv data file to dataframe
weather_df = pd.read_csv(weatherpy_file)
weather_df.head()

# Check point
# weather_df.dtypes

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,atuona,-9.80,-139.03,80.47,76,3,14.47,PF,1593400402
1,tshikapa,-6.42,20.80,70.21,53,20,2.84,CD,1593400402
2,athabasca,54.72,-113.29,57.92,87,100,10.18,CA,1593400402
3,karratha,-20.74,116.85,81.84,21,3,5.99,AU,1593400402
4,tuktoyaktuk,69.45,-133.04,48.20,81,20,9.17,CA,1593400402


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# configure gmaps
gmaps.configure(api_key=g_key)

# create list for location (lat, lng) and Humidity
locations = weather_df[["Lat","Lng"]]
humidity = weather_df["Humidity"]

# Plot heatmap
fig = gmaps.figure(layout={
        'width': '800px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'})

# Create heat layer
heat_layer= gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=max(weather_df["Humidity"]), point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
#Narrow down the DataFrame to find your ideal weather condition

# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph & Zero cloudiness.


fitting_weather_df = weather_df.loc[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70) &
                                   (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0)]

# Drop any rows that don't contain all three conditions.
fitting_weather_df = fitting_weather_df.dropna().reset_index(drop=True)

fitting_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,barra do corda,-5.50,-45.25,73.53,76,0,2.19,BR,1593400412
1,shitanjing,39.23,106.34,74.97,29,0,6.35,CN,1593400425
2,jinchang,38.50,102.17,79.27,22,0,4.59,CN,1593400427
3,idlib,35.93,36.63,75.20,83,0,5.82,SY,1593400430
4,tripoli,32.88,13.19,77.65,47,0,6.67,LY,1593400431
5,broome,-17.97,122.23,78.80,41,0,6.93,AU,1593400433
6,kosonsoy,41.26,71.55,75.20,25,0,4.70,UZ,1593400441
7,port said,31.26,32.28,73.40,83,0,6.93,EG,1593400232
8,tokur,53.13,132.90,73.87,35,0,8.25,RU,1593400444
9,shepsi,44.04,39.15,71.80,78,0,6.49,RU,1593400444


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = fitting_weather_df

# params dictionary
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key}

In [21]:
for index, row in hotel_df.iterrows():
    
    # value for latitude and logitude from the hotel_df
    lat= row["Lat"]
    lng= row["Lng"]
    
    # Add locaiton to params as google place search for Nearby Search request 
    params["location"] = f"{lat},{lng}"
    
    # Base url for nearby search request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make API request
    request = requests.get(base_url, params= params)
    
    #convert to json
    request_json = request.json()
    
    # check point - pretty print request_json data
    #pprint(request_json)
    
    # try-except to skip missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = request_json["results"][0]["name"]
    except: 
        pass
    
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,barra do corda,-5.50,-45.25,73.53,76,0,2.19,BR,1593400412,Hotel Diniz
1,shitanjing,39.23,106.34,74.97,29,0,6.35,CN,1593400425,NaN
2,jinchang,38.50,102.17,79.27,22,0,4.59,CN,1593400427,Jinchuan Hotel
3,idlib,35.93,36.63,75.20,83,0,5.82,SY,1593400430,Carlton Hotel
4,tripoli,32.88,13.19,77.65,47,0,6.67,LY,1593400431,Radisson Blu Al Mahary Hotel
5,broome,-17.97,122.23,78.80,41,0,6.93,AU,1593400433,Kimberley Sands Resort
6,kosonsoy,41.26,71.55,75.20,25,0,4.70,UZ,1593400441,Abdulloh Uyi
7,port said,31.26,32.28,73.40,83,0,6.93,EG,1593400232,فندق رستا بورسعيد Resta Port Said Hotel
8,tokur,53.13,132.90,73.87,35,0,8.25,RU,1593400444,NaN
9,shepsi,44.04,39.15,71.80,78,0,6.49,RU,1593400444,Otel' Chayka


In [23]:
# Drop missing hotel name columns
hotel_df = hotel_df.dropna()

# reset index of hotel_df
hotel_df.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,barra do corda,-5.50,-45.25,73.53,76,0,2.19,BR,1593400412,Hotel Diniz
1,jinchang,38.50,102.17,79.27,22,0,4.59,CN,1593400427,Jinchuan Hotel
2,idlib,35.93,36.63,75.20,83,0,5.82,SY,1593400430,Carlton Hotel
3,tripoli,32.88,13.19,77.65,47,0,6.67,LY,1593400431,Radisson Blu Al Mahary Hotel
4,broome,-17.97,122.23,78.80,41,0,6.93,AU,1593400433,Kimberley Sands Resort
5,kosonsoy,41.26,71.55,75.20,25,0,4.70,UZ,1593400441,Abdulloh Uyi
6,port said,31.26,32.28,73.40,83,0,6.93,EG,1593400232,فندق رستا بورسعيد Resta Port Said Hotel
7,shepsi,44.04,39.15,71.80,78,0,6.49,RU,1593400444,Otel' Chayka
8,hun,29.13,15.95,72.88,35,0,7.63,LY,1593400450,Alrwasi Hotel


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map

fig = gmaps.figure(layout={
        'width': '800px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'})

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='800px'))